In [2]:
!pip install pandas textblob openai


In [4]:
import pandas as pd


file_path = r"C:\Users\HP\Downloads\68b1acd44f393_Sample_Support_Emails_Dataset.csv"  # Make sure this file is in the same folder as your notebook
df = pd.read_csv(file_path)


df.head()


,sender,subject,body,sent_date
0,eve@startup.io,Help required with account verification,Do you support integration with third-party AP...,2025-08-19 00:58:09
1,diana@client.co,General query about subscription,"Hi team, I am unable to log into my account si...",2025-08-25 00:58:09
2,eve@startup.io,Immediate support needed for billing error,"Hello, I wanted to understand the pricing tier...",2025-08-20 12:58:09
3,alice@example.com,Urgent request: system access blocked,"Hi team, I am unable to log into my account si...",2025-08-21 21:58:09
4,eve@startup.io,Question: integration with API,"Despite multiple attempts, I cannot reset my p...",2025-08-20 04:58:09


In [6]:

keywords = ['support', 'query', 'request', 'help']


df_filtered = df[df['subject'].str.lower().str.contains('|'.join(keywords))]

df_filtered.reset_index(drop=True, inplace=True)
df_filtered.head()


,sender,subject,body,sent_date
0,eve@startup.io,Help required with account verification,Do you support integration with third-party AP...,2025-08-19 00:58:09
1,diana@client.co,General query about subscription,"Hi team, I am unable to log into my account si...",2025-08-25 00:58:09
2,eve@startup.io,Immediate support needed for billing error,"Hello, I wanted to understand the pricing tier...",2025-08-20 12:58:09
3,alice@example.com,Urgent request: system access blocked,"Hi team, I am unable to log into my account si...",2025-08-21 21:58:09
4,alice@example.com,Critical help needed for downtime,"Hi team, I am unable to log into my account si...",2025-08-18 08:58:09


In [8]:
def classify_priority(body):
    urgent_keywords = ['immediately', 'critical', 'asap', 'cannot access', 'urgent']
    body = body.lower()
    return 'Urgent' if any(word in body for word in urgent_keywords) else 'Not Urgent'

df_filtered['priority'] = df_filtered['body'].apply(classify_priority)
df_filtered[['subject', 'priority']].head()


C:\Users\HP\AppData\Local\Temp\ipykernel_1940\1520917399.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['priority'] = df_filtered['body'].apply(classify_priority)


,subject,priority
0,Help required with account verification,Not Urgent
1,General query about subscription,Not Urgent
2,Immediate support needed for billing error,Not Urgent
3,Urgent request: system access blocked,Not Urgent
4,Critical help needed for downtime,Not Urgent


In [12]:
def classify_priority(subject, body):
    urgent_keywords = ['immediately', 'critical', 'asap', 'cannot access', 'urgent', 'immediate', 'downtime']
    combined_text = f"{subject} {body}".lower()
    return 'Urgent' if any(word in combined_text for word in urgent_keywords) else 'Not Urgent'


df_filtered.loc[:, 'priority'] = df_filtered.apply(lambda row: classify_priority(row['subject'], row['body']), axis=1)


In [14]:
def classify_priority(subject, body):
    urgent_keywords = ['immediately', 'critical', 'asap', 'cannot access', 'urgent', 'immediate', 'downtime']
    combined_text = f"{subject} {body}".lower()
    return 'Urgent' if any(word in combined_text for word in urgent_keywords) else 'Not Urgent'


df_filtered.loc[:, 'priority'] = df_filtered.apply(lambda row: classify_priority(row['subject'], row['body']), axis=1)


In [16]:
def classify_priority(subject, body):
    urgent_keywords = ['immediately', 'critical', 'asap', 'cannot access', 'urgent', 'immediate', 'downtime']
    combined_text = f"{subject} {body}".lower()
    return 'Urgent' if any(word in combined_text for word in urgent_keywords) else 'Not Urgent'

# Apply the updated logic using .loc
df_filtered.loc[:, 'priority'] = df_filtered.apply(
    lambda row: classify_priority(row['subject'], row['body']),
    axis=1
)


df_filtered[['subject', 'priority']].head()


,subject,priority
0,Help required with account verification,Not Urgent
1,General query about subscription,Not Urgent
2,Immediate support needed for billing error,Urgent
3,Urgent request: system access blocked,Urgent
4,Critical help needed for downtime,Urgent


In [18]:
from textblob import TextBlob

def get_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    if polarity > 0.2:
        return 'Positive'
    elif polarity < -0.2:
        return 'Negative'
    else:
        return 'Neutral'


df_filtered.loc[:, 'sentiment'] = df_filtered['body'].apply(get_sentiment)


df_filtered[['subject', 'priority', 'sentiment']].head()




C:\Users\HP\AppData\Local\Temp\ipykernel_1940\1344834874.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, 'sentiment'] = df_filtered['body'].apply(get_sentiment)


,subject,priority,sentiment
0,Help required with account verification,Not Urgent,Neutral
1,General query about subscription,Not Urgent,Negative
2,Immediate support needed for billing error,Urgent,Positive
3,Urgent request: system access blocked,Urgent,Negative
4,Critical help needed for downtime,Urgent,Negative


In [20]:
def generate_reply(row):
    greeting = "Hi,"
    closing = "Let us know if you need any further assistance.\n\nBest regards,\nSupport Team"

    if row['sentiment'] == 'Negative':
        empathy = "We're sorry to hear you're experiencing difficulties. "
    elif row['sentiment'] == 'Positive':
        empathy = "Thank you for your message! "
    else:
        empathy = ""

    if row['priority'] == 'Urgent':
        urgency = "We’ll look into this on priority and get back to you as soon as possible. "
    else:
        urgency = "Our team will get back to you shortly. "

    return f"{greeting}\n\n{empathy}{urgency}{closing}"

# Apply the function
df_filtered.loc[:, 'ai_reply'] = df_filtered.apply(generate_reply, axis=1)

# ✅ Display the replies
df_filtered[['subject', 'priority', 'sentiment', 'ai_reply']].head()


C:\Users\HP\AppData\Local\Temp\ipykernel_1940\2723488738.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, 'ai_reply'] = df_filtered.apply(generate_reply, axis=1)


,subject,priority,sentiment,ai_reply
0,Help required with account verification,Not Urgent,Neutral,"Hi,\n\nOur team will get back to you shortly. ..."
1,General query about subscription,Not Urgent,Negative,"Hi,\n\nWe're sorry to hear you're experiencing..."
2,Immediate support needed for billing error,Urgent,Positive,"Hi,\n\nThank you for your message! We’ll look ..."
3,Urgent request: system access blocked,Urgent,Negative,"Hi,\n\nWe're sorry to hear you're experiencing..."
4,Critical help needed for downtime,Urgent,Negative,"Hi,\n\nWe're sorry to hear you're experiencing..."


In [22]:
df_filtered[['sender', 'subject', 'sent_date', 'priority', 'sentiment', 'ai_reply']].to_csv('email_assistant_output.csv', index=False)
